In [58]:
import requests
import os
import time
import pickle
import glob
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
# from tqdm.auto import tqdm
# from tqdm import tqdm_notebook
from tqdm import tqdm
# tqdm.pandas()

In [59]:
chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument(f"user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(
    options=chrome_options
)

In [2]:
def scroll_down_page(driver):
    SCROLL_PAUSE_TIME = 0.5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
    
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [3]:
def create_accounts():
    account = []
    with open("accounts.txt", "r", encoding='utf-8') as f:
        lines = f.readlines()
        for i in lines:
            tmp = dict()
            i = i.split()
            tmp["link"] = i[0]
            tmp["login"] = i[1]
            tmp["pass"] = i[2]
            accounts.append(tmp)
    return accounts

# Посты вконтакте

In [4]:
def auth_vk(driver, acc):
    link = acc.get("link")
    driver.get(url=link)
    # form_auth = driver.find_element
    input_login = driver.find_element('xpath','//*[@id="index_email"]')
    input_login.clear()
    input_login.send_keys(acc["login"])
    button_login = driver.find_element('xpath','//*[@id="index_login"]/div/form/button')
    button_login.click()
    time.sleep(2)
    input_pass = driver.find_element('xpath','//input[@name="password"]')        
    input_pass.clear()
    input_pass.send_keys(acc["pass"])
    button_login = driver.find_element('xpath','//button[@type="submit"]')
    button_login.click()
    time.sleep(2)

In [5]:
def get_vk_posts(driver, key_word):
    input_search = driver.find_element('xpath','//*[@id="ts_wrap"]/div/div/div/label/input')
    input_search.clear()
    input_search.send_keys(key_word)
    time.sleep(5)
    
    input_search.send_keys(Keys.ENTER)
    time.sleep(3)
    link_news = driver.find_element('xpath', '//*[@id="ui_rmenu_statuses"]')
    link_news.click()
    time.sleep(3)
    scroll_down_page(driver)
    time.sleep(1)
    # _post_content
    list_posts = driver.find_elements('class name', '_post_content')
    return list_posts

In [6]:
def get_info_vk_posts(list_posts):
    info_posts = []
    for post in tqdm(list_posts, total=len(list_posts)):
        soup = BeautifulSoup(post.get_attribute('outerHTML'), 'html.parser')
        name_group = soup.select_one(".PostHeaderTitle__authorName").text
        link_group = "https://vk.com"+soup.select_one(".PostHeaderTitle__authorLink").get('href')
        link_post = "https://vk.com"+soup.select_one(".PostHeaderSubtitle__link").get('href')
        date_post = soup.select_one(".PostHeaderSubtitle__item").text
        text_post = soup.select_one(".wall_post_text").text
        images_links = ''
        images = soup.select_one('.MediaGridContainerWeb--post')
        if images:
            images = images.findAll('img')
            images_links = '\n'.join([img['src'] for img in images])
        count_likes = soup.select_one('.PostButtonReactions__title').text
        count_reposts = soup.select('.PostBottomAction__count')[1].text
        count_views = soup.find('span', {'class' : '_views'}).text
        tmp = {
        "Название группы":name_group,
        "Ссылка группы":link_group,
        "Ссылка на пост":link_post,
        "Дата публикации поста":date_post,
        "Текст поста":text_post,
        "Картики в посте":images_links,
        "Количество лайков поста":count_likes,
        "Количество репостов поста":count_reposts,
        "Количество просмотров":count_views,
        }
        info_posts.append(tmp)
    return info_posts

In [7]:
def get_subs_vk_group(driver, link):
    driver.get(url=link)
    try:
        subs = driver.find_element('class name', 'group_friends_count')
        return subs.text
    except Exception as ex:
        try:
            el = driver.find_element('id','public_followers')
            subs = el.find_element('class name', 'header_count').text
            return ''.join(subs.split())
        except:
            return "Не группа или скрыто"
        return "Не группа или скрыто"


In [8]:
def create_cookie(accounts):
    for acc in accounts:
        if acc.get("link") == "https://vk.com/":
            driver = webdriver.Chrome(
                # executable_path="chromedriver.exe",
                options=chrome_options
            )
            link = acc.get("link")
            driver.get(url=link)
            input_login = driver.find_element('xpath','//*[@id="index_email"]')
            input_login.clear()
            input_login.send_keys(acc["login"])
            button_login = driver.find_element('xpath','//*[@id="index_login"]/div/form/button')
            button_login.click()
            time.sleep(2)
            input_pass = driver.find_element('xpath','//input[@name="password"]')        
            input_pass.clear()
            input_pass.send_keys(acc["pass"])
            button_login = driver.find_element('xpath','//button[@type="submit"]')
            button_login.click()
            time.sleep(2)
            name_cookie = os.path.join(r"cookie/vk", f"{acc['login']}_cookies_vk")
            
            with open(name_cookie, 'wb') as file:
                pickle.dump(driver.get_cookies(), file)
            driver.close()
            driver.quit()

# Посты Одноклассники

In [9]:
def auth_ok(driver, acc):
    link = acc.get("link")
    driver.get(url=link)
    # form_auth = driver.find_element
    time.sleep(1)
    input_login = driver.find_element('xpath','//*[@id="field_email"]')
    input_login.clear()
    input_login.send_keys(acc["login"])
    input_pass = driver.find_element('xpath','//*[@id="field_password"]')
    input_pass.clear()
    input_pass.send_keys(acc["pass"])
    form_buttons = driver.find_element('class name', 'login-form-actions')
    button_login = form_buttons.find_element('xpath','//input[@type="submit"]')
    button_login.click()

In [53]:
def get_ok_posts(driver, key_word):
    input_search = driver.find_element('name','st.query')
    input_search.clear()
    input_search.send_keys(key_word)
    time.sleep(1)
    input_search.send_keys(Keys.ENTER)
    time.sleep(1)
    link_news = driver.find_element('xpath', '//button[@id="tab-publications"]')
    link_news.click()
    time.sleep(1)
    scroll_down_page(driver)
    time.sleep(1)
    # _post_content
    list_posts = driver.find_elements('class name', 'feed')
    list_posts = [post.find_element('class name', 'media-text_a').get_attribute('href') for post in list_posts]
    return list_posts

In [109]:
def get_info_ok_posts(driver, posts_link):
    info_posts = []
    for post_link in tqdm(posts_link, total=len(list_posts)):
        driver.get(url=post_link)
        time.sleep(1)
        post = driver.find_element('class name', 'mlr_cnts')
        soup = BeautifulSoup(post.get_attribute('outerHTML'), 'html.parser')
        name_group = soup.select_one(".group-link").text
        link_group = "https://ok.ru"+soup.select_one(".group-link").get('href')
        link_post = post_link
        tmp = soup.find_all('div', {'class': 'ucard_add-info_i'})
        date_post = tmp[0].text
        subs_goup = tmp[1].text.replace('\xa0', '').split()[0]
        text_post = soup.select_one(".media-text_cnt_tx").text
        # images_links = ''
        images = soup.find_all('img')
        images_links = '\n'.join([img['src'] for img in images])
        # print(images)
        # if images:
        #     images = images.findAll('img')
        #     images_links = '\n'.join([img['src'] for img in images])
        widget_list = soup.select_one('.mlr_bot').select_one('.widget-list')
        count_comments = widget_list.select_one('li[data-widget-item-type="comment"]').select_one('.widget_count').text
        count_reposts = widget_list.select_one('li[data-widget-item-type="reshare"]').select_one('.widget_count').text
        count_likes = widget_list.select_one('li[data-widget-item-type="like"]').select_one('.widget_count').text
        tmp = {
        "Название группы":name_group.strip(),
        "Ссылка группы":link_group.strip(),
        "Количество подписчиков":subs_goup.strip(),
        "Ссылка на пост":link_post.strip(),
        "Дата публикации поста":date_post.strip(),
        "Текст поста":text_post.strip(),
        "Картики в посте":images_links,
        "Количество лайков поста":count_likes.strip(),
        "Количество репостов поста":count_reposts.strip(),
        "Количество комментариев":count_comments.strip(),
        }
        info_posts.append(tmp)
    return info_posts

In [108]:
accounts = []



accounts = create_accounts()
for i, acc in enumerate(accounts):
    if acc.get("link") == "https://vk.com/":
        pass
        # auth_vk(driver, acc)
        # time.sleep(5)
        # list_posts = get_vk_posts(driver, 'кибрарий')
        # info_posts = get_info_vk_posts(list_posts)
        # res = pd.DataFrame(info_posts)
        # res.to_excel("vk_posts.xlsx")
        # for rowi in tqdm(res.itertuples(), total=res.shape[0]):
        #     i = getattr(rowi, 'Index')
        #     url = getattr(rowi, '_2')
        #     subs = str(get_subs_vk_group(driver, url))
        #     res.loc[i, 'Количество подписчиков группы'] = subs
        # res.to_excel("vk_posts.xlsx")
    elif acc.get("link") == "https://ok.ru/":
        pass
        # auth_ok(driver, acc)
        # list_posts = get_ok_posts(driver, 'кибрарий')
        # info_posts = get_info_ok_posts(driver, list_posts)
        # res = pd.DataFrame(info_posts)
        # res.to_excel("ok_posts.xlsx")
time.sleep(5)
# driver.close()
# driver.quit()

100%|██████████████████████████████████████████████████████████████████████████████████| 51/51 [02:08<00:00,  2.53s/it]
